In [ ]:
!pip install -U scikit-learn

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

# Load the dataset
df = pd.read_csv('intent_dataset.csv')

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Intent'], test_size=0.2, random_state=42)

# Create a pipeline with a text vectorizer and a support vector classifier
model = make_pipeline(TfidfVectorizer(), SVC(probability=True))

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

    Farewell       1.00      0.56      0.71        27
       Greet       0.50      0.65      0.56        17
     Inquiry       0.65      1.00      0.79        13

    accuracy                           0.68        57
   macro avg       0.72      0.73      0.69        57
weighted avg       0.77      0.68      0.69        57



In [ ]:
def classify_intent(text, model, threshold=0.7):
    # Predict the probability distribution over the classes
    probs = model.predict_proba([text])[0]
    # Get the maximum probability and its corresponding class
    confidence = max(probs)
    intent = model.classes_[probs.argmax()]

    # Check if the confidence meets the threshold
    if confidence < threshold:
        return "NLU fallback: Intent could not be confidently determined"
    else:
        return f"Intent: {intent}, Confidence: {confidence:.2f}"


In [ ]:
test_texts = [
    "Hello, how are you?",
    "I need to find the nearest gas station.",
    "Goodbye, see you tomorrow!",
    "What is the time?",
    "Could you help me with the directions?"
]

for text in test_texts:
    result = classify_intent(text, model)
    print(f"Input: '{text}' - Output: {result}")


Input: 'Hello, how are you?' - Output: Intent: Greet, Confidence: 1.00
Input: 'I need to find the nearest gas station.' - Output: Intent: Inquiry, Confidence: 0.82
Input: 'Goodbye, see you tomorrow!' - Output: Intent: Farewell, Confidence: 1.00
Input: 'What is the time?' - Output: Intent: Inquiry, Confidence: 1.00
Input: 'Could you help me with the directions?' - Output: Intent: Inquiry, Confidence: 0.73


In [5]:
from joblib import dump

# Save the model to disk
dump(model, 'trained_intent_classifier.joblib')


['trained_intent_classifier.joblib']

In [6]:
from joblib import load

# Load the model from disk
model = load('trained_intent_classifier.joblib')

# Example usage
result = classify_intent("Hello, how are you?", model)
print(result)


Intent: Greet, Confidence: 1.00
